In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd

In [41]:
user='G'

if user=='S':
    mandat_dep=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/liste_mandat.csv")#on récupère le csv de la liste des mandats
    df_depu=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/listeDepute_complete.csv")

if user=='G':
    mandat_dep=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandatdef.csv") 
    df_depu=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/listeDepute_complete.csv")

<h1> I. Base mandat

### Extraction de la date de début du mandat

In [42]:
debut_mandat=[]
for dates in mandat_dep["Mandat"]:
    dates=str(dates).split()
    debut_mandat.append(' '.join(dates[1:4]))
    
mandat_dep["Debut Mandat"]=np.array(debut_mandat)
mandat_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007


### Etablissement de l'âge du député lors de son mandat

In [75]:
df_depu.head(2)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort
0,0,Ludovic Pajot,18 novembre 1993,NaN,NaN
1,1,Typhanie Degois,6 janvier 1993,NaN,NaN


In [76]:
mandat_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007


In [45]:
#De même que dans le fichier Webscrapp, cette ligne va poser problème
age_dep=mandat_dep.set_index('Nom').join(df_depu[["Nom","Né le"]].set_index('Nom'), on='Nom')
mandat_dep.reset_index()
mois={'janvier':1,'février':2, 'mars':3, 'avril':4, 'mai':5, 'juin':6, 'juillet':7, 'août':8, 'septembre':9, 'octobre':10, 'novembre':11, 'décembre':12}
def transtype_date(x):
    x=str(x)        
    date_texte=x.split() 
    if len(date_texte)>1 :
        if  date_texte[1] in (mois.keys()):
            if date_texte[0]=="1er":
                date_texte[0]='1'
            date_texte[1]=str(mois[date_texte[1]])
            date_nombre='/'.join(date_texte)
            return pd.to_datetime(date_nombre)
    else:
        return pd.NaT


age_dep["Date de naissance"]=age_dep["Né le"].apply(lambda x: transtype_date(x))
age_dep["Date debut mandat"]=age_dep["Debut Mandat"].apply(lambda x: transtype_date(x))
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat
Nom,,,,,,,,,,
Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,1964-11-11,2002-06-19
Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,1964-11-11,2007-06-20


In [46]:
age_dep['age depute']=age_dep['Date debut mandat']-age_dep['Date de naissance']#on crée la variable age au debut du mandat, en jour
age_dep['age en année']=age_dep['age depute'].apply(lambda x: float(x.days / 365.25))#et on la transforme en années
#age_dep=age_dep[['Département','Groupe', 'Législature', 'Mandat', 'Régime politique', 'age en année','Date debut mandat']]

In [77]:
age_dep.head(2)

,index,Nom,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat,age depute,age en année
0,0,Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,1964-11-11,2002-06-19,13734 days,37.601643
1,1,Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,1964-11-11,2007-06-20,15561 days,42.603696


On remarque que certains âges sont négatifs! C'est toujours à cause des problèmes que l'on a à cause des députés ayant le même nom. Pour de plus amples explications et des exemples, aller voir le fichier 'analyses nan'.

In [48]:
age_dep[age_dep["age en année"]<0].head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat,age depute,age en année
Nom,,,,,,,,,,,,
Paul Giacobbi,0,Corse,Radical et radical-socialiste,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,21 octobre 1945,4 juin 1957,1957-04-06,1945-10-21,-4185 days,-11.457906
Paul Giacobbi,0,Corse,Radical et radical-socialiste,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,21 octobre 1945,4 juin 1957,1957-04-06,1945-10-21,-4185 days,-11.457906


In [74]:
df_depu.groupby("Nom").count().sort_values(by="Né le").tail(5)

,Unnamed: 0,Né le,Département de naissance,Département de mort
Nom,,,,
Jean Rousseau,4,4,4,4
Pierre Dumas,9,9,9,9
Pierre Charles,9,9,9,9
Pierre Leroy-Beaulieu,9,9,9,9
Louis Brunet,9,9,9,9


<h1> II.Table député

<h3> Nombre de mandat par député

In [51]:
nb_mandat=age_dep.groupby("Nom").count()
nb_mandat=pd.DataFrame(nb_mandat).rename(columns={"age en année":"nombre de mandat"})
nb_mandat.head()

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat,age depute,nombre de mandat
Nom,,,,,,,,,,,,
"(Louis, Camille, Alexandre, Marie Dubochet",1,1,0,1,1,1,1,1,1,1,1,1
"Aaron, Jean-François Crassous",1,1,1,1,1,1,1,1,1,1,1,1
Abbas Gueye,1,1,1,1,1,1,1,1,1,1,1,1
Abbes Moulessehoul,1,1,1,1,1,1,1,1,1,1,1,1
Abdallah Tebib,1,1,1,1,1,1,1,1,1,1,1,1


In [62]:
df_depu_b=df_depu.join(nb_mandat["nombre de mandat"],on="Nom",lsuffix=" ")
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat
2343,2343,Serge Franchis,10 septembre 1933,Yonne,Inconnu,1.0
13619,13619,"Antoine, François Delpierre",3 février 1764,Vosges,Vosges,2.0
7020,7020,Alfred-Léon Gérault-Richard,11 octobre 1860,Sarthe,Var,4.0
1939,1939,Daniel Reiner,17 janvier 1941,Inconnu,Inconnu,1.0
2220,2220,Joseph Pinard,7 avril 1936,Doubs,Inconnu,1.0


In [63]:
premier_mandat=age_dep.groupby("Nom")["age en année"].min()
premier_mandat.head()

Nom
(Louis, Camille, Alexandre, Marie Dubochet    45.689254
Aaron, Jean-François Crassous                 49.267625
Abbas Gueye                                   37.470226
Abbes Moulessehoul                            38.811773
Abdallah Tebib                                60.766598
Name: age en année, dtype: float64

In [64]:
age_dep=age_dep.reset_index()

In [65]:
premier_mandat_dpt=age_dep.groupby("Nom")[["Département","Nom"]].head(1)
premier_mandat_dpt=pd.DataFrame(premier_mandat_dpt).rename(columns={"Département":"premier dpt"}).set_index("Nom")
premier_mandat_dpt.head(3)

,premier dpt
Nom,
Damien Meslot,Territoire-de-Belfort
Christine Cloarec-Le Nabour,Ille-et-Vilaine
Pascal Terrasse,Ardèche


<h3> Caractéristiques du premier mandat

In [66]:
df_depu_b=df_depu_b.join(premier_mandat,on="Nom",lsuffix="")
df_depu_b=df_depu_b.join(premier_mandat_dpt,on="Nom",lsuffix="")

In [67]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt
10997,10997,"Eugène, Jean Dezeimeris",29 novembre 1799,Dordogne,Paris,2.0,42.770705,Dordogne
9501,9501,"Bernard, Camille Collas",2 mars 1819,Gironde,Inconnu,1.0,30.272416,Gironde
11827,11827,"Firmin, Louis Aubergé",16 décembre 1788,Oise,Inconnu,2.0,59.348392,Seine-et-Marne
6469,6469,Ferri de Ludre,11 août 1870,Seine,Paris,4.0,31.463381,Meurthe-et-Moselle
1736,1736,Didier Bariani,16 octobre 1943,Allier,Inconnu,3.0,34.381930,Paris


In [68]:
df_depu_b["élection locale"]=(" "+df_depu_b["Département de naissance"])==df_depu_b["premier dpt"]

In [69]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt,élection locale
10992,10992,"Christian, Léon Dumas",14 décembre 1799,Seine,Paris,2.0,45.765914,Charente-Inférieure,False
10148,10148,Charles Desmoutiers,30 janvier 1810,Nord,Nord,3.0,38.228611,Nord,True
4941,4941,Cerf Lurie,31 mars 1897,Hérault,Hérault,1.0,61.664613,Hérault,True
15861,15861,Gilbert Bruys de Charly,15 septembre 1744,Inconnu,Inconnu,2.0,66.058864,Saône-et-Loire,False
4292,4292,Clément Lavergne,23 septembre 1905,Cantal,Essonne,3.0,40.076660,Cantal,True


<h1> III. Base département

In [17]:
dep_possible=list(mandat_dep['Département'].unique())#une liste de tous les départements
dep_possible.remove(" Département d'élection inconnu")
dep_possible.remove(np.nan)

In [18]:
dep_possible[:3]

[' Territoire-de-Belfort', ' Ille-et-Vilaine', ' Ardèche']

In [19]:
def pourcentageTerroir(basedep) :#une fonction qui donne le pourcentage de mandats occupés par des députés nés dans leur département d'élection
    basedep_1=basedep[basedep['Département']==' '+basedep['Département de naissance']]
    nbreterroir=basedep_1[basedep_1['Département de naissance']!='Inconnu']['Département'].count()
    i=basedep[basedep['Département de naissance']!='Inconnu']['Département'].count()
    if i!=0:
        pourcTerroir=nbreterroir/i
    else:
        return np.nan
    return pourcTerroir

In [20]:
df=age_dep.reset_index()

In [21]:
dept_naiss=age_dep.set_index('Nom').join(df_depu[["Nom","Né le", "Département de naissance"]].set_index('Nom'), on='Nom')
dept_naiss=dept_naiss.reset_index()
dept_naiss.head()

,Nom,Département,Groupe,Législature,Mandat,Régime politique,age en année,Date debut mandat,Né le,Département de naissance
0,Damien Meslot,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,37.601643,2002-06-19,11 novembre 1964,Inconnu
1,Damien Meslot,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,42.603696,2007-06-20,11 novembre 1964,Inconnu
2,Damien Meslot,Territoire-de-Belfort,Union pour un mouvement populaire,XIVe législature,Du 20 juin 2012 au 20 juin 2017,Cinquième République - Assemblée nationale,47.605749,2012-06-20,11 novembre 1964,Inconnu
3,Christine Cloarec-Le Nabour,Ille-et-Vilaine,La République en Marche,XVe législature (Législature en cours),Depuis le 21 juin 2017 (mandat en cours),Cinquième République - Assemblée nationale,NaN,NaT,28 octobre 1964,Inconnu
4,Pascal Terrasse,Ardèche,Socialiste,XIe législature,Du 1er juin 1997 au 18 juin 2002,Cinquième République - Assemblée nationale,32.197125,1997-01-06,26 octobre 1964,Gard


In [22]:

w,x,y,z,u=[],[],[],[],[]
for dep in dep_possible :
    w+=[df[df['Département']==dep]['age en année'].mean()]#on prend la moyenne d'age dans le département
    x+=[len(df[df['Département']==dep])]#on compte le nombre total de mandat dans le département
    y+=[len(df[df['Département']==dep]['Nom'].unique())]#Puis le nombre de députés différent par dpt
    z+=[x[-1]/y[-1]] #Ceci est le nombre moyen de mandat par député
    u+=[pourcentageTerroir(dept_naiss[dept_naiss["Département"]==dep])]
    
    
carac_dep_bis=np.array([w, x, y, z,u])#on crée un dataframe avec les informations
table_depmt = pd.DataFrame(carac_dep_bis, index=['age moyen des deputes', 'nombre_mandat', 'nombre député','nombre moyen de mandats par député', 'locaux'],columns=dep_possible)
table_depmt.T.sample(10)

,age moyen des deputes,nombre_mandat,nombre député,nombre moyen de mandats par député,locaux
Vendée,49.478806,351.0,147.0,2.387755,0.711765
Yonne,48.862315,357.0,147.0,2.428571,0.505519
Départements de l'Empire français aujourd'hui en Italie,47.717454,89.0,89.0,1.000000,0.000000
Alpes-de-Haute-Provence,55.715872,9.0,7.0,1.285714,0.000000
Territoire français des Affars et des Issas,39.978097,4.0,3.0,1.333333,0.000000
Bas-Rhin,47.690881,424.0,197.0,2.152284,0.750000
Soudan-Niger,43.079398,4.0,3.0,1.333333,0.000000
Comores,51.297833,15.0,4.0,3.750000,0.000000
Tarn,47.211667,329.0,149.0,2.208054,0.662921
Hauts-de-Seine,54.076791,184.0,67.0,2.746269,0.000000


In [81]:
table_depute=df_depu_b
table_depute.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/table_depute.csv")

In [30]:
table_depmt.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/table_dpmt.csv")